## LBC Demonstration

#### Starting up the system

In [93]:
import sys
#sys.path.append('C:/Users/Triphase.WS191692/Documents/Tripy')
#sys.path.append('/home/triphase')
import tripy as trp
%matplotlib inline

In [2]:
from pathlib import Path

path = Path.cwd()

In [3]:
path

WindowsPath('C:/Users/Triphase.WS191692/Documents/Tripy')

In [4]:
import paramiko

In [ ]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

In [ ]:
test = ssh.connect('172.22.151.35', '22','triphase', 'Tr1Ph4s3')
test

In [ ]:
stdin, stdout, stderr = ssh.exec_command('ls -l')

In [ ]:
stdout.read()

In [ ]:
ftp = ssh.open_sftp()

In [ ]:
ftp.chdir('/home/triphase')

In [ ]:
cwd=ftp.getcwd()
print(ftp.getcwd())

In [ ]:
ftp.listdir()

In [ ]:
#data_d = ftp.chdir('/home/triphase/model/triphase/src')
cwd=ftp.getcwd()
path = Path.cwd()
path    
#ftp.get("modbusData.db","modbusData.db",callback=None)

In [55]:
import socket as sckt
def find_targets(default_ip = '192.168.0.255', default_port = 8282):
    target_names = []
    target_ips = []
    s = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
    #s.setsockopt(sckt.SOL_SOCKET, sckt.SO_BROADCAST, 1)
    s.connect((default_ip, default_port))
    s.settimeout(0.1)
    string = 'GETHOSTNAME\0'
    out_data = string.encode('us-ascii')
    #broadcast_ip = default_ip.rsplit('.', 1)[0] + '.255'
    #s.sendto(out_data, (broadcast_ip, default_port))
    s.sendto(out_data, (default_ip, default_port))
    blah = True
    while blah:
        try:
            in_data, addr = s.recvfrom(256)
            target_names.append(in_data.decode())
            target_ips.append(addr[0])
            blah = False
        except sckt.error as er:
            print(er)
            break
    return {'ips' : target_ips, 'names' : target_names}
    #return in_data

In [58]:
find_targets(default_ip = '172.22.151.35',default_port = 8282)

timed out


{'ips': [], 'names': []}

In [50]:
def isOpen(ip,port):
    s = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
    try:
        s.connect((ip, int(port)))
        s.shutdown(2)
        return True
    except:
        return False

In [51]:
isOpen('172.22.151.35', 8282)

True

In [13]:
HOST = '172.22.151.35'    # The remote host
PORT = 50007               # The same port as used by the server
with sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    s.settimeout(0.1)
    s.sendall(b'Hello, world')
    data = s.recv(1024)
print('Received', repr(data))

Received b'Hello, world'


In [94]:
def find_targets(default_ip = '192.168.0.255', default_port = 8282):
    target_names = []
    target_ips = []
    s = sckt.socket(sckt.AF_INET, sckt.SOCK_DGRAM)
    s.setsockopt(sckt.SOL_SOCKET, sckt.SO_BROADCAST, 1)
    s.settimeout(0.1)
    string = 'GETHOSTNAME\0'
    out_data = string.encode('us-ascii')
    broadcast_ip = default_ip.rsplit('.', 1)[0] + '.255'
    #print(broadcast_ip)
    #s.sendto(out_data, (broadcast_ip, default_port))
    s.sendto(out_data, ('172.22.151.35', default_port))
    while True:
        try:
            in_data, addr = s.recvfrom(256)
            #print(in_data)
            #print(addr)
            target_names.append(in_data.decode())
            target_ips.append(addr[0])
        except sckt.error:
            break
    return {'ips' : target_ips, 'names' : target_names}

In [95]:
import socket as sckt

find_targets(default_ip = '172.22.151.35',default_port = 8282)

{'ips': ['172.22.151.35'], 'names': ['3p-han-1']}

In [80]:
sckt.gethostbyname('sustainablecharging.nl')
#sckt.gethostbyaddr('172.22.151.35')
sckt.gethostname()

'WS191692'

In [96]:
#m = trp.Model(model = 'PM15A30I60F06_afAC3_vsAC3_csDC1', user = 'piet', host = '172.22.151.35')
m = trp.Model(model = 'C:/Users/Triphase.WS191692/Documents/Tripy/models/piet/src/PM15A30I60F06_afAC3_vsAC3_csDC1', user = 'piet', host = '172.22.151.35')

Select(description='Select one of the available Triphase realtime targets: ', options={}, value=None)

IndexError: list index out of range

If no specific realtime target is chosen, the code above will look for Triphase realtime targets on the local network and show you a list of options. You can select the realtime target by clicking on the name. 
You can specify the target by simply adding _host = '192.168.0.101'_ to the code above between the round brackets.

In [ ]:
m.start()

#### Providing an overview of the available signals and parameters:

**Signals** are outputs of the system to our python interface. **Parameters** are inputs from the python interface to the system.

In [ ]:
m.signals

In [ ]:
m.parameters

#### Interfacing with the active front:

In [ ]:
U_grid = m.capture_data('u_out_af', samples = 4000)

By default the amount of samples taken, is set to 1000. The decimation is set to 1 by default. You can alter both by adding them to the code above: U_grid = m.capture_data('u_out_af', samples = 2000, decimation = 2).

In [ ]:
for i in range(1:1000):
    m.capture_data('u_out_af', samples = 4000)
U_grid.plot()

Once data is captured, you can do much more than simply plotting the data. Python comes with the _Numpy_ package, which is great for doing computations. For instance I can use the data I've captured to calculate the harmonic content.

In [ ]:
#Determine the frequency
freq = U_grid.frequency()
print(freq)
#Determining the harmonic content

H = U_grid.harmonic_content(freq,[1,3,5,7,9,11,13,15,17])
H

In [ ]:
U_grid.plot(har = [1,3,5,7,9,11,13,15,17,19,21,23,25,27])

You can capture multiple inputs at once instead of capturing them one by one.

In [ ]:
grid = m.capture_data(['u_out_af', 'i_out_af'], 2000, 1)

In [ ]:
grid.plot()

In [ ]:
I_grid = m.capture_data(['i_out_af'], 4000, 1)

In [ ]:
I_grid.plot(har = [1,3,5,7,9,11,13,15,17,19,21,23,25,27])

In [ ]:
m.parameters

In [ ]:
m.signals

In [ ]:
m.set_parameter('External_param', 1)

In [ ]:
m.set_parameter('enable_af', 1)

In [ ]:
U_dcbus = m.capture_data('interface_af/u_dcbus', 4000, 10)

In [ ]:
U_dcbus.plot()

In [ ]:
m.set_parameter('u_dcbus_SP', 680)

In [ ]:
U_dcbus = m.capture_data('interface_af/u_dcbus', 4000, 10)

In [ ]:
U_dcbus.plot()

In [ ]:
m.set_parameter('enable_af', 0)

In [ ]:
m.stop()